<a href="https://colab.research.google.com/github/pachterlab/scATAK/blob/main/scATAK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# install minimap to find peaks
!wget --quiet --show-progress "https://github.com/lh3/minimap2/releases/download/v2.24/minimap2-2.24_x64-linux.tar.bz2"
!tar -jxvf minimap2-2.24_x64-linux.tar.bz2
!cp minimap2-2.24_x64-linux/minimap2 /usr/local/bin

minimap2-2.24_x64-l 100%[===================>]   2.04M  --.-KB/s    in 0.06s   
minimap2-2.24_x64-linux/
minimap2-2.24_x64-linux/LICENSE.txt
minimap2-2.24_x64-linux/k8
minimap2-2.24_x64-linux/README-js.md
minimap2-2.24_x64-linux/paftools.js
minimap2-2.24_x64-linux/minimap2
minimap2-2.24_x64-linux/README.md
minimap2-2.24_x64-linux/cookbook.md
minimap2-2.24_x64-linux/minimap2.1
minimap2-2.24_x64-linux/NEWS.md


In [8]:
!minimap2

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer (preferrable for PacBio)
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minimizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM[,NUM] chaining/alignment bandwidth and long-join bandwidth [500,20000]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all m

In [9]:
!wget --quiet --show-progress "http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz"

--2022-09-30 17:06:13--  http://ftp.ensembl.org/pub/release-104/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 881211416 (840M) [application/x-gzip]
Saving to: ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’

Homo_sapiens.GRCh38 100%[===================>] 840.39M  23.9MB/s    in 4m 24s  

2022-09-30 17:10:37 (3.18 MB/s) - ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’ saved [881211416/881211416]



In [10]:
!time minimap2 \
-d ref.mmi \
Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz

tcmalloc: large alloc 1073741824 bytes == 0x6a698000 @  0x7effc6ef72a4 0x414d2d 0x406c48 0x4129cf 0x7effc66f06db 0x7effc641961f
tcmalloc: large alloc 2147483648 bytes == 0x27d99e000 @  0x7effc6ef72a4 0x406fee 0x4129cf 0x7effc66f06db 0x7effc641961f
[M::mm_idx_gen::127.246*1.68] collected minimizers
[M::mm_idx_gen::161.853*1.73] sorted minimizers
[M::main::190.503*1.54] loaded/built the index for 194 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 194
[M::mm_idx_stat::192.955*1.53] distinct minimizers: 100159079 (38.79% are singletons); average occurrences: 5.540; average spacing: 5.586; total length: 3099750718
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -d ref.mmi Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
[M::main] Real time: 194.609 sec; CPU: 296.872 sec; Peak RSS: 11.262 GB

real	3m15.225s
user	4m14.967s
sys	0m42.510s


In [ ]:
!minimap2 -ax sr -t2 ref.mmi \
   <(paste \
     <(zcat $r1_fastq) \
     <(zcat $r2_fastq) \
     | awk -v blen="$BLEN" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   <(paste \
     <(zcat $r3_fastq) \
     <(zcat $r2_fastq) \
     | awk -v blen="$BLEN" '{if(NR%4==1) header=$1; if(NR%4==2) print header"_"substr($2, 0, blen)"_\n"$1; if(NR%4==3 || NR%4==0) print $1;}') \
   > peaks.sam

In [11]:
# once you get data, put up a minimal dataset on github